#### Norwegian Spell Corrector
#### Author : Anugraha Sinha
#### Email : anugraha[dot]sinha[at]gmail[dot]com

We use the **Project Gutenberg EBook of The Adventures of Sherlock Holmes by Sir Arthur Conan Doyle** as the base document to build a large vocab of english words. We can use some other vocab source as well (May be like wikipedia article dump etc.)

In [1]:
import re
from collections import Counter

##### Function to convert all words to lower case

In [2]:
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

##### Building a frequency information for all words

In [3]:
all_words = Counter(words(open('big_doc_for_spelling_correction.txt').read()))

##### example : check frequency for a random word

In [4]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

#### example : check top 10 most common words

In [5]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

Building the edit_one function, so as to calculate

1 step correction based on following operations
1. delete
2. insert
3. replace
4. transpose


In [6]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

Building edit_two function, which is basically checking execution of edit_one twice

In [7]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

Function to check if the given words are known words (present in vocab) or not.

In [8]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

**possible_correction** based on 
1. if the word is already present in vocab
2. if the word is edit_one (1 edit distance) away from a known word (set of possible words)
3. if the word is edit_two (2 edit distance) away from a known word (set of possible words)

In [9]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

**prob**

probability value of the suggested word, based on the vocab

In [10]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

##### Example :
Checking the possible correction for mis-spelled word "emfasize" or "emfasis"

In [12]:
print(possible_corrections("emfasize"))
print(possible_corrections("emfasis"))

{'emphasize'}
{'emphasis'}


##### Example:

edit_one execution

In [13]:
print(known(edits_one("monney")))

{'money', 'monkey'}


In [14]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money', 'monkey'}


#### Example:

correction for word "monney"

In [15]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money', 'monkey'}


##### Example:

execution for calculating probability if the word is correct or incorrect.

In [16]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


Final function to execute spell correction

In [17]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [18]:
# test spell check
print(spell_check("elobarate"))

Did you mean elaborate?
